In [ ]:
# import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd

In [ ]:
# Define the dataset class for a custom dataset

class CustomDataset(Dataset):
    
    def __init__(self, data_frame, transform=None):
        self.data = data_frame
        self.transform = transform
#         
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        obs = self.data.iloc[index, 0 : 2].values
        label = self.data.iloc[index, 2]
        
        if self.transform is not None:
            obs = self.transform(obs)
            
        return obs, label

In [4]:
# Create the toy data

data_ = pd.DataFrame({'X1' : [-15, -15 ,-15.2, -15.4, -15.52, -14.89, -14.5, -16.222, 16, 15.1, 15.2, 15.4, 15.52, 15.155, 15.02, 15.222,
                              -15, -15.66 ,-15.92, -15.12, -15, -14, -14, -16, 16, 14.1, 14.2, 15.4, 15.52, 15.155, 17.02, 16.222],
                   'X2' : [5, 5, 6.5, 6, 7.021, 6.03, 5.05, 6.016, -6, -5, -6.01, -7.011, -5.021, -7, -6.05, -6.016,
                           6, 6, 5.5, 6, 6.021, 5.03, 5.05, 7.016, -6, -5, -5.01, -6.011, -6.021, -7, -5.05, -6.216], 
                   'label' : [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
                              1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]})
# Plot the toy data
plt.figure(figsize = (5, 5))
plt.scatter(data_['X1'], data_['X2'])
plt.xlabel('feature 1')
plt.ylabel('feature 2')
plt.xlim(-20, 20)
plt.ylim(-20, 20)

In [ ]:
train_dataset = CustomDataset(data_, transform = None)

In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size = 16, shuffle = True, num_workers = 2)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class linear_model(nn.Module):
  def __init__(self):
    super(linear_model, self).__init__()
    self.fc1 = nn.Linear(2, 1)
    nn.init.xavier_normal_(self.fc1.weight)
    self.fc2 = nn.Linear(2, 1)
    nn.init.xavier_normal_(self.fc2.weight)


  def forward(self, x):
    x = x.view(-1, 2)
    x1 = torch.sigmoid( self.fc1(x) )  
    x2 = torch.sigmoid( self.fc2(x) )  
  
    
    return x1, x2
  
model = linear_model()  

In [ ]:
#Script for generating fancy inner product matrix:

sigma = 0.1 #HIGHER VALUES OF SIGMA SEEM TO PERFORM BETTER
M = torch.zeros([2, 2])

M_tensor = M.view(1,1,28,28)

for i_1 in range(28):
    for i_2 in range(28):
        for j_1 in range(28):
            for j_2 in range(28):
                M_tensor[i_1,i_2,j_1,j_2] = np.exp( -((i_1 - j_1)**2 + (i_2 - j_2)**2) / (2 * (sigma)**2) )

M = M_tensor.view([28*28,28*28])

In [ ]:
# Define loss function and optimizer
import torch.optim as optim

loss_function = nn.BCELoss(reduction = 'mean') # sums all outputs and divides by total data points

optimizer = optim.Adam(model.parameters(), lr = 0.1)

class custom_loss(torch.nn.Module):

  def __init__(self):
    super(custom_loss,self).__init__()

  def forward(self, mat1, mat2):
  ## Absolute dot product of weights
    l = (torch.mm(mat1, mat2.t()))**2

  ## Fancy dot product if weights
    #  l = (torch.mm(mat1, torch.mm(M, mat2.t())))**2
    return l

class regularizer(torch.nn.Module):

  def __init__(self):
    super(regularizer,self).__init__()

  def forward(self, weig):
 
  ## l-1 norm
    # l = torch.sum(torch.abs(weig))
 
  # l2 norm
    l = torch.sqrt(torch.sum(weig * weig))
    return l

sl = custom_loss()    
reg = regularizer()

In [ ]:
# A list of models

models_forloss = [model.fc1.weight, model.fc2.weight]

In [1]:
n_epochs = 5

l_r = 100 # tunable
l_o = 10000 # tunable

for epochs in range(n_epochs):
 
  running_loss = 0
  
  for batch in trainloader:
    data, targets = batch

    optimizer.zero_grad()

    output = model(data.float()) 
 
    loss = loss_function(output[0], targets.view(-1, 1).float()) + loss_function(output[1], targets.view(-1, 1).float()) + l_o * sl.forward(model.fc1.weight, model.fc2.weight) + l_r * reg.forward(model.fc1.weight) + l_r * reg.forward(model.fc2.weight)
 
  
    loss.backward()

    optimizer.step()

    running_loss += loss.item()
  print(running_loss)

In [2]:
# Test performance on the entire test set

correct = 0
total = 0
with torch.no_grad():
  for data in trainloader:
    images, labels = data
    outputs = model(images.float())
    predicted = outputs[0] > 0.5
    total += labels.size(0)
    matches = 0
    for i in range(len(predicted)):
      if predicted[i].item() == labels[i].item():
        matches += 1
    correct += matches
  print('Accuracy of the network on the entire data set is : %d %%' %(100 * correct/ total))

In [3]:
# Visualising the learned weights

x_s1 = list(range(-20, 25, 5))
y_s1 = []
y_s2 = []
for el in x_s1:
  y_s1.append((-model.fc1.weight[0][0].item()/ model.fc1.weight[0][1].item()) * el)

for el in x_s1:
  y_s2.append((-model.fc2.weight[0][0].item()/ model.fc2.weight[0][1].item()) * el)

plt.plot(x_s1, y_s1, color = 'red') 
plt.plot(x_s1, y_s2, color = 'orange') 

plt.scatter(data_['X1'], data_['X2'])
plt.xlabel('feature 1')
plt.ylabel('feature 2')
plt.xlim(-20, 20)
plt.ylim(-20, 20)